<a href="https://colab.research.google.com/github/587855/ToxicCommentsClassification/blob/main/Basis_ToxicComments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cud

In [ ]:
import os
import re
import shutil
import string
import keras
import tensorflow as tf
import pandas as pd
from datasets import load_dataset

In [ ]:
# Load Civil Comments from Huggingface
civil_comments = load_dataset('google/civil_comments')
df_civil = pd.DataFrame(civil_comments['train'])
print('Civil Comments shape:', df_civil.shape)
print(df_civil[['text', 'toxicity']].head())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.73k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/187M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1804874 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/97320 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/97320 [00:00<?, ? examples/s]

Civil Comments shape: (1804874, 8)
                                                text  toxicity
0  This is so cool. It's like, 'would you want yo...  0.000000
1  Thank you!! This would make my life a lot less...  0.000000
2  This is such an urgent design problem; kudos t...  0.000000
3  Is this something I'll be able to install on m...  0.000000
4               haha you guys are a bunch of losers.  0.893617


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text.strip()

# Apply cleaning
df_civil['text_clean'] = df_civil['text'].astype(str).apply(clean_text)

# Create binary toxicity label (1 if toxicity >= 0.5, else 0)
df_civil['toxic'] = (df_civil['toxicity'] >= 0.5).astype(int)

# Split into train and test
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_civil['text_clean'], df_civil['toxic'], test_size=0.2, random_state=42, stratify=df_civil['toxic'])

print('Train size:', len(train_texts))
print('Test size:', len(test_texts))

Train size: 1443899
Test size: 360975


In [ ]:
# Check the first rows of the dataframe
pd.set_option('display.max_colwidth', None)
print(df_civil[['text_clean', 'toxic']].head())

                                                                                                    text_clean  \
0                 this is so cool its like would you want your mother to read this really great idea well done   
1  thank you this would make my life a lot less anxietyinducing keep it up and dont let anyone get in your way   
2                          this is such an urgent design problem kudos to you for taking it on very impressive   
3                            is this something ill be able to install on my site when will you be releasing it   
4                                                                          haha you guys are a bunch of losers   

   toxic  
0      0  
1      0  
2      0  
3      0  
4      1  


# Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 20000
max_len = 150
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

print('Padded train shape:', X_train_pad.shape)
print('Padded test shape:', X_test_pad.shape)

Padded train shape: (1443899, 150)
Padded test shape: (360975, 150)


In [ ]:
# Check original text, tokenized, and padded to verify that the text has been tokenized correctly.
for i in range(2):
    print("Original:", train_texts.iloc[i])
    print("Tokenized:", X_train_seq[i])
    print("Padded:", X_train_pad[i])
    print('-'*70)

Original: cowards are against any and all war because they are cowards
Tokenized: [6246, 14, 153, 70, 4, 28, 293, 74, 21, 14, 6246]
Padded: [6246   14  153   70    4   28  293   74   21   14 6246    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]
------------------------------------------------------------
Original: we the people continue to s

# Baseline Models

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=10000)
X_train_tfidf = tfidf.fit_transform(train_texts)
X_test_tfidf = tfidf.transform(test_texts)

# Logistic Regression
lr = LogisticRegression(max_iter=200) # class_weight='balanced'
lr.fit(X_train_tfidf, train_labels)
lr_preds = lr.predict(X_test_tfidf)

print('Logistic Regression:')
print('Accuracy:', accuracy_score(test_labels, lr_preds))
print('Precision:', precision_score(test_labels, lr_preds))
print('Recall:', recall_score(test_labels, lr_preds))
print('F1:', f1_score(test_labels, lr_preds))

# Multinomial Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_tfidf, train_labels)
nb_preds = nb.predict(X_test_tfidf)

print('Naive Bayes:')
print('Accuracy:', accuracy_score(test_labels, nb_preds))
print('Precision:', precision_score(test_labels, nb_preds))
print('Recall:', recall_score(test_labels, nb_preds))
print('F1:', f1_score(test_labels, nb_preds))

Logistic Regression:
Accuracy: 0.9450682180206386
Precision: 0.776865580198505
Recall: 0.4392558977379014
F1: 0.5611985217641462
Naive Bayes:
Accuracy: 0.924864602811829
Precision: 0.9560899111343439
Recall: 0.06335954550178405
F1: 0.1188434048083171


# LSTM

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

embedding_dim = 128

lstm_model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model
history = lstm_model.fit(
    X_train_pad, train_labels,
    epochs=3,
    batch_size=256,
    validation_split=0.1
)

lstm_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/3
5077/5077 ━━━━━━━━━━━━━━━━━━━━ 54s 10ms/step - accuracy: 0.9235 - loss: 0.2635 - val_accuracy: 0.9475 - val_loss: 0.1384
Epoch 2/3
5077/5077 ━━━━━━━━━━━━━━━━━━━━ 49s 10ms/step - accuracy: 0.9489 - loss: 0.1345 - val_accuracy: 0.9491 - val_loss: 0.1349
Epoch 3/3
5077/5077 ━━━━━━━━━━━━━━━━━━━━ 48s 9ms/step - accuracy: 0.9524 - loss: 0.1228 - val_accuracy: 0.9496 - val_loss: 0.1323


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 150, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,828,421 (29.86 MB)

 Trainable params: 2,609,473 (9.95 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,218,948 (19.91 MB)

In [ ]:
# Evaluate the LSTM model
lstm_preds = lstm_model.predict(X_test_pad)
lstm_preds_bin = (lstm_preds >= 0.5).astype(int)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('LSTM:')
print('Accuracy:', accuracy_score(test_labels, lstm_preds_bin))
print('Precision:', precision_score(test_labels, lstm_preds_bin))
print('Recall:', recall_score(test_labels, lstm_preds_bin))
print('F1:', f1_score(test_labels, lstm_preds_bin))

11281/11281 ━━━━━━━━━━━━━━━━━━━━ 33s 3ms/step
LSTM:
Accuracy: 0.9500630237551076
Precision: 0.7715273255522717
Recall: 0.5335504208958326
F1: 0.6308416956788859


# GRU

In [ ]:
from tensorflow.keras.layers import GRU

gru_model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len),
    GRU(64, return_sequences=False),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

gru_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the GRU model
gru_history = gru_model.fit(
    X_train_pad, train_labels,
    epochs=3,
    batch_size=256,
    validation_split=0.1
)
gru_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/3
5077/5077 ━━━━━━━━━━━━━━━━━━━━ 49s 9ms/step - accuracy: 0.9324 - loss: 0.2161 - val_accuracy: 0.9495 - val_loss: 0.1308
Epoch 2/3
5077/5077 ━━━━━━━━━━━━━━━━━━━━ 47s 9ms/step - accuracy: 0.9520 - loss: 0.1257 - val_accuracy: 0.9494 - val_loss: 0.1333
Epoch 3/3
5077/5077 ━━━━━━━━━━━━━━━━━━━━ 47s 9ms/step - accuracy: 0.9562 - loss: 0.1133 - val_accuracy: 0.9487 - val_loss: 0.1362


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 150, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,791,941 (29.72 MB)

 Trainable params: 2,597,313 (9.91 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,194,628 (19.82 MB)

In [ ]:
# Evaluate the GRU model
gru_preds = gru_model.predict(X_test_pad)
gru_preds_bin = (gru_preds >= 0.5).astype(int)

print('GRU:')
print('Accuracy:', accuracy_score(test_labels, gru_preds_bin))
print('Precision:', precision_score(test_labels, gru_preds_bin))
print('Recall:', recall_score(test_labels, gru_preds_bin))
print('F1:', f1_score(test_labels, gru_preds_bin))

11281/11281 ━━━━━━━━━━━━━━━━━━━━ 33s 3ms/step
GRU:
Accuracy: 0.9490934275226817
Precision: 0.7798346225660177
Recall: 0.5063913811618803
F1: 0.6140468789380829


In [ ]:
# Example sentence
sentence = "I hate you, you are stopid"

# Clean the sentence (optional, but recommended for consistency)
sentence_clean = clean_text(sentence)

# Tokenize and pad
seq = tokenizer.texts_to_sequences([sentence_clean])
pad = pad_sequences(seq, maxlen=max_len, padding='post', truncating='post')

# Predict with the GRU model
pred_gru = gru_model.predict(pad)[0][0]
print("GRU Model")
# Interpret the result
if pred_gru >= 0.5:
    print(f'"{sentence}" is classified as TOXIC (score: {pred_gru:.3f})')
else:
    print(f'"{sentence}" is classified as NON-TOXIC (score: {pred_gru:.3f})')

# Predict with the LSTM model
lstm_preds = lstm_model.predict(pad)[0][0]
print("LSTM Model")
# Interpret the result
if lstm_preds >= 0.5:
    print(f'"{sentence}" is classified as TOXIC (score: {lstm_preds:.3f})')
else:
    print(f'"{sentence}" is classified as NON-TOXIC (score: {lstm_preds:.3f})')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
GRU Model
"I hate you, you are a wank" is classified as TOXIC (score: 0.593)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
LSTM Model
"I hate you, you are a wank" is classified as NON-TOXIC (score: 0.460)
